# Build an Agentic RAG Service

Setup an agent service that can interact with a tool service (containing RAG tools over annual reports).

In this notebook, we:
- Setup our indexes and query engine tools
- Define our multi-agent framework
  - A message queue.
  - An agentic orchestrator.
  - A tools service containing our query engine tools. This will act as a remote executor for tools
  - Define meta-tools for our agents. These will make calls to the tools service instead of executing directly
  - Our agent services. These wrap existing llama-index agents
  - Put all this into a local launcher, to simulate one task passing through the system at a time.

In [1]:
import nest_asyncio

nest_asyncio.apply()

import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai_api_key = os.getenv('OPENAI_API_KEY')

## Load Data

In [2]:
# !mkdir reports
# !wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2020/executive-summary-2020.pdf -O ./reports/executive_summary_2020.pdf
# !wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2021/executive-summary-2021.pdf -O ./reports/executive_summary_2021.pdf
# !wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2022/executive-summary-2022.pdf -O ./reports/executive_summary_2022.pdf
# !wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2023/executive-summary-2023.pdf -O ./reports/executive_summary_2023.pdf

In [6]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [24]:
query_engine_tools = []
for filename in os.listdir("reports"):
    if filename.endswith(".pdf"):
        file_path = os.path.join("reports", filename)

        # Load data
        documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
        print(f"Loaded {len(documents)} documents from {filename}")
        print(filename[:-4])  

        # Build index
        splitter = SentenceSplitter(chunk_size=512)
        nodes = splitter.get_nodes_from_documents(documents)
        vector_index = VectorStoreIndex(nodes)

        # # Persist index
        # persist_dir = f"./storage/{filename[:-4]}"
        # vector_index.storage_context.persist(persist_dir=persist_dir)
        # executive_summaries[int(filename[:-4])] = vector_index

        query_engine = vector_index.as_query_engine(similarity_top_k=10)
        query_engine_tool = QueryEngineTool(
            query_engine=query_engine,
            metadata=ToolMetadata(
                name=f"{filename[:-4]}",  # Construct name without extension
                description=(
                    f"Provides information about the U.S. government financial report {filename[:-4]}"
                ),
            ),
        )
        query_engine_tools.append(query_engine_tool)

Loaded 10 documents from executive_summary_2022.pdf
executive_summary_2022
Loaded 10 documents from executive_summary_2023.pdf
executive_summary_2023
Loaded 11 documents from executive_summary_2021.pdf
executive_summary_2021
Loaded 11 documents from executive_summary_2020.pdf
executive_summary_2020


In [25]:
query_engine_tools[1].metadata

ToolMetadata(description='Provides information about the U.S. government financial report executive_summary_2023', name='executive_summary_2023', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

## Setup Agents

Now that we've defined the query tools, we can wrap these under a `ToolService`.

In [30]:
from llama_agents import (
    AgentService,
    ToolService,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.llms.openai import OpenAI


# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=OpenAI(model="gpt-4o-mini")),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=query_engine_tools,
    running=True,
    step_interval=0.5,
)

# define meta-tools
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in query_engine_tools
]

# define agent worker and agent service
worker = FunctionCallingAgentWorker.from_tools(
    meta_tools,
    llm=OpenAI(),
)
agent = worker.as_agent()
agent_server = AgentService(
    agent=agent,
    message_queue=message_queue,
    description="Used to answer questions the U.S. government financial report",
    service_name="financial_report_agent",
)


## Launch agent 

With our services, orchestrator, control plane, and message queue defined, we can test our llama-agents network by passing in single messages, and observing the results.

This is an excellent way to test, iterate, and debug your llama-agents system.

In [29]:
import logging

# change logging level to enable or disable more verbose logging
logging.getLogger("llama_agents").setLevel(logging.INFO)

In [31]:
from llama_agents import LocalLauncher

## Define Launcher
launcher = LocalLauncher(
    [agent_server, tool_service],
    control_plane,
    message_queue,
)

In [32]:
# What is the U.S. government's net operating cost?
# What are the total revenues reported by the U.S. government?
# What are the total assets reported by the U.S. government?
# What is the total national debt reported by the U.S. government?
query_str = "What is the U.S. government's net operating cost from 2020 to 2023?"
result = launcher.launch_single(query_str)

INFO:llama_agents.message_queues.simple - Consumer AgentService-a2334ae1-9bd6-4101-bb6b-34d28bad79f4: financial_report_agent has been registered.
INFO:llama_agents.message_queues.simple - Consumer ToolService-7cdb43b6-e4d7-4f48-bd6b-5b0d7059a33e: default_tool_service has been registered.
INFO:llama_agents.message_queues.simple - Consumer 82df6212-0067-4c8d-ae69-6013b045b3a9: human has been registered.
INFO:llama_agents.message_queues.simple - Consumer ControlPlaneServer-78619b8d-0510-49fe-b612-1ea8e8eb9786: control_plane has been registered.
INFO:llama_agents.services.agent - financial_report_agent launch_local
INFO:llama_agents.message_queues.base - Publishing message to 'control_plane' with action 'new_task'
INFO:llama_agents.message_queues.simple - Launching message queue locally
INFO:llama_agents.services.agent - Processing initiated.
INFO:llama_agents.services.tool - Processing initiated.
INFO:llama_agents.message_queues.base - Publishing message to 'financial_report_agent' with a

In [33]:
print(result)

From 2020 to 2023, the U.S. government's net operating costs were as follows:

- 2020: $3.8 trillion
- 2021: $3.1 trillion (a decrease of $746.5 billion or 19.4%)
- 2022: $4.2 trillion (an increase of $1.1 trillion)
- 2023: $3.4 trillion

This shows a significant fluctuation in net operating costs over these years.


In [34]:
query_str = "Compare and contrast the annual net operating cost from 2020 to 2023, then give an analysis"
result = launcher.launch_single(query_str)

INFO:llama_agents.message_queues.simple - Consumer AgentService-a2334ae1-9bd6-4101-bb6b-34d28bad79f4: financial_report_agent has been registered.
INFO:llama_agents.message_queues.simple - Consumer ToolService-7cdb43b6-e4d7-4f48-bd6b-5b0d7059a33e: default_tool_service has been registered.
INFO:llama_agents.message_queues.simple - Consumer eef8f14f-4b16-4986-bd9b-4ce7130251d5: human has been registered.
INFO:llama_agents.message_queues.simple - Consumer ControlPlaneServer-78619b8d-0510-49fe-b612-1ea8e8eb9786: control_plane has been registered.
INFO:llama_agents.services.agent - financial_report_agent launch_local
INFO:llama_agents.message_queues.base - Publishing message to 'control_plane' with action 'new_task'
INFO:llama_agents.message_queues.simple - Launching message queue locally
INFO:llama_agents.services.agent - Processing initiated.
INFO:llama_agents.services.tool - Processing initiated.
INFO:llama_agents.message_queues.base - Publishing message to 'financial_report_agent' with a

In [35]:
print(result)

The U.S. government's annual net operating costs from 2020 to 2023 are as follows: 

- 2020: $3.8 trillion (an increase of $2.4 trillion from the previous year)
- 2021: $3.1 trillion (a decrease of $746.5 billion, or 19.4%)
- 2022: $4.2 trillion (an increase of $1.1 trillion)
- 2023: $3.4 trillion

### Analysis:
1. **2020**: The net operating cost was significantly high at $3.8 trillion, largely due to the economic impact of the COVID-19 pandemic, which led to increased government spending on relief measures.
2. **2021**: There was a notable decrease to $3.1 trillion, indicating a reduction in spending as the economy began to recover and pandemic-related expenses decreased.
3. **2022**: The costs surged again to $4.2 trillion, possibly due to inflationary pressures and ongoing recovery efforts, as well as increased spending in various sectors.
4. **2023**: The net operating cost decreased to $3.4 trillion, suggesting a stabilization in spending and a potential return to more sustainabl